In [1]:
from KGNET import KGNET

/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kgnet=KGNET(KG_endpointUrl='http://206.12.98.118:8890/sparql',KG_NamedGraph_IRI='https://dblp2022.org',KG_Prefix='dblp2022')

## visualize graph schema

In [3]:
types_df=kgnet.getKGNodeEdgeTypes(write_to_file=True,prefix='dblp2022')
types_df=types_df[(~types_df["object"].str.endswith("_Obj")) | (types_df["object"].str.endswith("publishedIn_Obj"))]
g=kgnet.visualizeKG(types_df,Notebook=True,
                    width="100%",height="300px")
g.show('nx.html')

nx.html


## Training Pipeline

In [ ]:
taskId,modelId,trainMetadata=kgnet.train_GML(operatorType=KGNET.GML_Operator_Types.NodeClassification,
                                             GNNMethod=KGNET.GNN_Methods.RGCN,
                                             targetNodeType="dblp2022:Publication",labelNodeType="dblp2022:publishedIn_Obj")

### Query task Info from KGMeta

In [22]:
kgnet.KGMeta_Governer.getGMLTaskBasicInfoByID(taskId)

,property,Value
0,Task ID,25
1,Task Name,NodeClassifier>dblp2022>dblp2022:Publication>d...
2,Task Type,kgnet:type/nodeClassification
3,KG name,dblp2022
4,KG description,Subgraph of dblp 2022-03 KG that contains only...
5,Target Edge,https://dblp.org/rdf/schema#publishedIn
6,Target Node Type,dblp2022:Publication
7,Date Created,"19/08/2023, 14:46:30"
8,# Trained Models,20


### Query Task Trained Models Info from KGMeta
<li> The green color indicates the best model to choose.</li>
<li> The orange color indicates the model selected by KGNET Inference query optimizer which balance between infernce time and model accuracy. </li>

In [23]:
df,styler_df=kgnet.KGMeta_Governer.getGMLTaskModelsBasicInfoByID(taskId)
styler_df

,Property,Model ID,GNN Method,Training Memory GB.,Training Time Sec.,Test Acc Score,Inference Time,Date Created
0,Model #1,109,RGCN,0.956001,38.150000,23,51,"28/08/2023, 22:12:31"
1,Model #2,110,G_SAINT,1.533730,352.210000,31,15,"29/08/2023, 05:14:25"
2,Model #3,83,G_SAINT,0.927505,538.690000,94,108,"21/08/2023, 15:38:02"
3,Model #4,85,G_SAINT,1.025944,379.460000,72,13,"21/08/2023, 15:57:26"
4,Model #5,106,G_SAINT,5.585407,371.450000,29,16,"25/08/2023, 16:18:31"
5,Model #6,113,RGCN,1.084667,43.370000,23,0,"29/08/2023, 13:41:20"
6,Model #7,82,G_SAINT,1.054363,516.560000,93,18,"21/08/2023, 14:29:39"
7,Model #8,91,G_SAINT,0.995323,285.370000,72,12,"21/08/2023, 18:49:41"
8,Model #9,96,ShaDowSAINT,5.985710,462.340000,23,19,"23/08/2023, 17:59:59"
9,Model #10,98,RGCN,2.652660,27.270000,23,20,"23/08/2023, 21:17:38"


## Inference Pipeline
write the SPARQL-ML inference query then execute it. KGNet will choose the model that match your task.

In [29]:
query = """
            prefix dblp2022:<https://dblp.org/rdf/schema#>
            prefix kgnet:<http://kgnet/>
            select ?Publication ?Title ?dblp_Venue ?Pred_Venue
            from <https://dblp2022.org>
            where
            {
            ?Publication a dblp2022:Publication .
            ?Publication ?NodeClassifier ?Pred_Venue .            
            ?Publication  dblp2022:publishedIn ?dblp_Venue .
            ?Publication  dblp2022:title ?Title .            
            ?NodeClassifier a <kgnet:types/NodeClassifier>.
            ?NodeClassifier <kgnet:targetNode> dblp2022:Publication.
            ?NodeClassifier <kgnet:labelNode> dblp2022:publishedIn_Obj.
            }
            limit 100
        """


In [32]:
import datetime
startT = datetime.datetime.now()
res_df,inferenceQueryMetadata=kgnet.executeSPARQLMLInferenceQuery(query)
print("The task selected model ID is:",inferenceQueryMetadata["model_id"])
print('time=',datetime.datetime.now()-startT)

The task selected model ID is: 94
time= 0:00:16.175151


In [33]:
highlighted_df=res_df.style.apply(kgnet.utils.highlightDiferrentRowValues, subset=['dblp_Venue','Pred_Venue'], axis=1)
highlighted_df

,Publication,Title,dblp_Venue,Pred_Venue
0,"""https://dblp.org/rec/conf/ctrsa/Bultel22""","""CCA Secure A Posteriori Openable Encryption in the Standard Model.""","""CT-RSA""","""CT-RSA"""
1,"""https://dblp.org/rec/conf/ctrsa/CaoSCCFW22""","""Lattice-Based Fault Attacks on Deterministic Signature Schemes of ECDSA and EdDSA.""","""CT-RSA""","""CT-RSA"""
2,"""https://dblp.org/rec/conf/ctrsa/Chevallier-Mames22""","""A Pairing-Free Signature Scheme from Correlation Intractable Hash Function and Strong Diffie-Hellman Assumption.""","""CT-RSA""","""CT-RSA"""
3,"""https://dblp.org/rec/conf/ctrsa/Chi-DominguezR22""","""Fully Projective Radical Isogenies in Constant-Time.""","""""","""CT-RSA"""
4,"""https://dblp.org/rec/conf/ctrsa/CidIR22""","""FASTA - A Stream Cipher for Fast FHE Evaluation.""","""CT-RSA""","""CT-RSA"""
5,"""https://dblp.org/rec/conf/ctrsa/CuiHWW22""","""Integral Attacks on Pyjamask-96 and Round-Reduced Pyjamask-128.""","""CT-RSA""","""CT-RSA"""
6,"""https://dblp.org/rec/conf/ctrsa/FouotsaP22""","""A New Adaptive Attack on SIDH.""","""CT-RSA""","""CT-RSA"""
7,"""https://dblp.org/rec/conf/ctrsa/Rosie22""","""Adaptively Secure Laconic Function Evaluation for NC1.""","""""","""CT-RSA"""
8,"""https://dblp.org/rec/conf/ctrsa/WuX22""","""More Accurate Geometric Analysis on the Impact of Successful Decryptions for IND-CCA Secure Ring/Mod-LWE/LWR Based Schemes.""","""CT-RSA""","""CT-RSA"""
9,"""https://dblp.org/rec/conf/padl/2022""","""Practical Aspects of Declarative Languages - 24th International Symposium, PADL 2022, Philadelphia, PA, USA, January 17-18, 2022, Proceedings""","""Lecture Notes in Computer Science""","""Lecture Notes in Computer Science"""


In [28]:
inferenceQueryMetadata['candidateSparqlQuery']

'PREFIX dblp2022: <https://dblp.org/rdf/schema#>\nPREFIX kgnet: <http://kgnet/>\nSELECT ?Publication  ?Title  ?dblp_Venue \n sql:getKeyValue_v2(?Publication,?Pred_Venue_dic) as ?Pred_Venue \n from <https://dblp2022.org> \nWHERE { \n?Publication <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> dblp2022:Publication .\n{SELECT sql:getNodeClass_v2("http://206.12.99.65:64647/gml_inference/mid/94","""{"model_id" : 94, "named_graph_uri" : "https://dblp2022.org", "sparqlEndpointURL" : "http://206.12.98.118:8890/sparql", "dataQuery" : ["PREFIX dblp2022: <https://dblp.org/rdf/schema#> PREFIX kgnet: <http://kgnet/> SELECT ?s ?p ?o   from <https://dblp2022.org>  where { ?s ?p ?o {  SELECT ?s  ?Title  ?dblp_Venue from <https://dblp2022.org>  WHERE {  ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> dblp2022:Publication . ?s dblp2022:publishedIn ?dblp_Venue . ?s dblp2022:title ?Title . ?s dblp2022:numberOfCreators \'3\' . }   LIMIT 100 } filter(!isBlank(?o)). }"] , "targetNodesQuery" : "PREFIX 